In [1]:
import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn
import pandas as pd
from pandas import Series
import os
import zipfile

C:\Users\smhrd\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
path_dir = './soundSource/'
file_list = os.listdir(path_dir)

In [3]:
file_list

['000-너의 모든 순간-성시경.mp3',
 '001-Monologue-테이(Tei).mp3',
 '002-혼자서 걸어요 (Prod. by 나얼)-태연 (TAEYEON).mp3',
 '003-Say I Love You-우디(Woody).mp3',
 '004-그댄 행복에 살텐데 (2022)-최유리.mp3',
 '005-해요 (2022)-#안녕.mp3',
 '006-마지막 사랑-신예영.mp3',
 '007-다정히 내 이름을 부르면-경서예지.mp3',
 '008-그때 그 순간 그대로 (그그그)-WSG워너비 (가야G).mp3',
 '009-모든 날, 모든 순간 (Every day, Every Moment)-폴킴(Paul Kim).mp3',
 '010-취중고백-김민석 (멜로망스).mp3',
 '011-나의 X에게-경서.mp3',
 '012-사랑은 늘 도망가-임영웅.mp3',
 '013-내가 아니라도-주호.mp3',
 '014-드라마-아이유(IU).mp3',
 '015-우린 그렇게 사랑해서-강민경.mp3',
 '016-그대를 알고-송하예.mp3',
 '017-잘가요-주호.mp3',
 '018-떠나보낼 준비해 둘걸 그랬어-임한별.mp3',
 '019-팡파레-다비치.mp3',
 '020-니 번호가 뜨는 일-이예준.mp3',
 '021-아픈 나를 (Prod. By 나얼)-성시경.mp3',
 '022-Christmas Tree-V.mp3',
 '023-눈이 오잖아(Feat.헤이즈)-이무진.mp3',
 '024-사랑해도 될까요-양다일.mp3',
 '025-어른-Sondia.mp3',
 '026-희재-성시경.mp3',
 '027-Wonder Why-경서.mp3',
 '028-진심이었던 사람만 바보가 돼-권진아.mp3',
 '029-어떻게 이별까지 사랑하겠어, 널 사랑하는 거지-AKMU(악뮤).mp3',
 '030-밤편지-아이유(IU).mp3',
 '031-밤하늘의 별을(2020)-경서.mp3',
 '032-다시 사랑한다면 (니글니글 버터플라이)-김필(Kim Feel).mp3',


In [4]:
def normalize(x, axis=0):
        return sklearn.preprocessing.minmax_scale(x, axis=axis) 

In [5]:
import scipy
from scipy import stats

In [6]:
dfs=[]
# 비어있는 DataFrame생성
df_mfcc = pd.DataFrame()
# 비어있는 list 생성
mfcc_list1 = []
mfcc_list2 = []
for i in range(len(file_list)):
    y, sr = librosa.load(path_dir+file_list[i])   # 파일 for문으로 돌리기
    
    # length ==> len(y) 즉 음악을 숫자로 읽어온 것의 길이. ()
    # new
    leng= int(len(y)/sr)  # 이거 하면 음악의 길이가 나옵니다.   189초로 바꾸고
    leng2=int(leng/2)    # 얘는 음악의 중간 시작 점을 나타냅니다.  절반인 94로 바뀌고
    n_start = (leng2-30)*22050     # 34에서 34* 컴퓨터가 인식하는 1초단위
    n_end = (leng2+30)*22050       # 94에서 컴퓨터가 인식하는 1초단위
    n_3 = int(22050*30)  # 1.5초 간격을 나타냅니다.

    D = np.abs(librosa.stft(y, n_fft=2048, hop_length=512))
    DB = librosa.amplitude_to_db(D, ref=np.max)
    S = librosa.feature.melspectrogram(y=y, sr=sr)
    S_DB = librosa.amplitude_to_db(S, ref=np.max)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    zero_crossings = librosa.zero_crossings(y=y, pad=False)
    y_harm, y_perc = librosa.effects.hpss(y)
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    frames = range(len(spectral_centroids))
    t = librosa.frames_to_time(frames)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    chromagram = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)    
    spectral_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rms = librosa.feature.rms(y=y)   

    filename = file_list[i]
    tempo1 = tempo
    zero_crossing_rate_mean = zero_crossings.mean()
    zero_crossing_rate_var = zero_crossings.var()
    y_harm_mean=y_harm.mean()
    y_harm_var= y_harm.var()
    y_perc_mean=y_perc.mean()
    y_perc_var=y_perc.var()
    spectral_centroid_mean=spectral_centroids.mean()
    spectral_centroid_var=spectral_centroids.var()
    rolloff_mean = spectral_rolloff.mean()
    rolloff_var = spectral_rolloff.var()
    chroma_stft_mean=chromagram.mean()
    chroma_stft_var=chromagram.var()
    spectral_bandwidth_mean = spectral_bw.mean()
    spectral_bandwidth_var = spectral_bw.var()
    label = file_list[i]
    rms_mean = np.mean(rms)
    rms_var = np.var(rms)
    
    for j in range(0, 2):
        mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)

        mfcc_normalized = normalize(mfcc, axis=1)
        globals()['mfcc'+str(j)] = mfcc_normalized

        mfcc_mean = mfcc_normalized.mean()
        #globals()['mfcc_mean'+str(j)] = mfcc_mean

        mfcc_var = mfcc_normalized.var()
                    #globals()['mfcc_var'+str(j)] = mfcc_var
        mfcc_list1.append(mfcc_mean)
        mfcc_list2.append(mfcc_var)

    dfs.append({'filename':filename,
                'tempo1':tempo1,
                'zero_crossing_rate_mean':zero_crossing_rate_mean,
                'zero_crossing_rate_var':zero_crossing_rate_var,
                'rms_mean':rms_mean,
                'rms_var':rms_var,
                'y_harm_mean':y_harm_mean,
                'y_harm_var':y_harm_var,
                'y_perc_mean':y_perc_mean,
                'y_perc_var':y_perc_var,
                'spectral_centroid_mean':spectral_centroid_mean,
                'spectral_centroid_var':spectral_centroid_var,
                'rolloff_mean':rolloff_mean,
                'rolloff_var':rolloff_var,
                'chroma_stft_mean':chroma_stft_mean ,
                'chroma_stft_var':chroma_stft_var ,
                'spectral_bandwidth_mean':spectral_bandwidth_mean ,
                'spectral_bandwidth_var':spectral_bandwidth_var ,
                #'spectral_flatness':spectral_flatness,
                'label':label})  
    
       

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.3972125  -0.3446988  -0.3152522  ...  0.07544409  0.06478367
  0.05103798] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[ 0.03369916  0.0175124   0.00925462 ... -0.5029648  -0.40534174
 -0.3269308 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.3012247  -0.4344468  -0.43721265 ...  0.22663328 -0.04052886
 -0.14085904] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[ 0.344322    0.32355452  0.34374222 ... -0.09661157 -0.27420285
 -0.31485045] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.28534603 -0.30607298 -0.15617844 ... -0.33018923 -0.38853827
 -0.45988044] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.0535713  -0.05880979 -0.06531064 ... -0.06441864  0.07096763
  0.26175788] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[ 0.1469186   0.24971585  0.2505145  ...  0.00579141 -0.04286658
 -0.15275191] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.14577594 -0.138268   -0.1528521  ...  0.2760711   0.24861239
  0.25609013] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[ 0.04068397  0.04196926  0.04003004 ... -0.12039768 -0.12604395
 -0.12697008] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[0.17523491 0.20511962 0.23265065 ... 0.10087645 0.09461756 0.07826858] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[ 0.07478236  0.06744182  0.06264514 ... -0.52918416 -0.52829295
 -0.45386648] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[ 0.38183787  0.35547024  0.41341397 ... -0.22476941 -0.2141139
 -0.21309566] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) 

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.06562292 -0.06827477 -0.06974623 ... -0.04066162 -0.04092386
 -0.04847493] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.05430297 -0.05925266 -0.06942011 ... -0.1426801  -0.24555174
 -0.25275606] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.11198501 -0.10677275 -0.07429191 ... -0.06126758 -0.05824992
 -0.06267543] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((

...

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[ 0.12206099 -0.02228728 -0.25740716 ...  0.42616108  0.4936813
  0.49502054] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[0.43898678 0.26265368 0.15403086 ... 0.29667464 0.2892541  0.2825811 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.03263364 -0.01460247 -0.12473889 ...  0.14573725  0.14659147
  0.14041685] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) 

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.19252084 -0.12518653 -0.11833126 ...  0.03444249  0.15671161
  0.19026104] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[ 0.1140132   0.05931205  0.09462287 ... -0.52547866 -0.5271937
 -0.52512234] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[0.47450423 0.4009967  0.39677218 ... 0.27306968 0.2715952  0.25928047] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) 

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.18948524 -0.16155648 -0.16815944 ... -0.08742018 -0.06287614
 -0.07895882] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.0446441  -0.04274452 -0.05180402 ...  0.05258647  0.07964268
  0.10992707] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-1.0132139e-02 -8.2184616e-03 -1.6088763e-02 ...  3.3226450e-05
  6.1567816e-06  2.5095651e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.29273158 -0.26586223 -0.2073756  ... -0.15870179 -0.18919653
 -0.1880909 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.16286592 -0.10862017 -0.01030705 ... -0.01932644 -0.00036239
 -0.00473547] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.25690693 -0.17899281 -0.44844717 ...  0.03777236  0.02906588
  0.07938469] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((

C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[ 0.05779781  0.07603472  0.09705567 ... -0.07190578  0.03969801
 -0.17372689] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.18885766 -0.07721042 -0.08948178 ...  0.07939375  0.01207996
  0.01050226] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((n_3)*j) : n_start+((n_3)*(j+1))], sr=sr)
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7476\441828557.py:54: FutureWarning: Pass y=[-0.16865374 -0.1722145  -0.19274332 ...  0.29005998  0.28763917
  0.28305423] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y[n_start+((

In [7]:
dfs_df = pd.DataFrame(dfs)

In [8]:
dfs_df

,filename,tempo1,zero_crossing_rate_mean,zero_crossing_rate_var,rms_mean,rms_var,y_harm_mean,y_harm_var,y_perc_mean,y_perc_var,spectral_centroid_mean,spectral_centroid_var,rolloff_mean,rolloff_var,chroma_stft_mean,chroma_stft_var,spectral_bandwidth_mean,spectral_bandwidth_var,label
0,000-너의 모든 순간-성시경.mp3,135.999178,0.063051,0.059076,0.206933,0.014392,0.000041,0.042217,0.000158,0.005620,1478.566950,5.113177e+05,3055.470936,2.830697e+06,0.266660,0.090366,1802.617495,273552.279734,000-너의 모든 순간-성시경.mp3
1,001-Monologue-테이(Tei).mp3,78.302557,0.123248,0.108058,0.219556,0.010052,0.000003,0.033471,0.000331,0.011100,2448.858859,6.861721e+05,5066.694181,2.592118e+06,0.344431,0.088892,2412.710045,167487.247350,001-Monologue-테이(Tei).mp3
2,002-혼자서 걸어요 (Prod. by 나얼)-태연 (TAEYEON).mp3,135.999178,0.081354,0.074735,0.145015,0.006357,0.000002,0.020258,-0.000002,0.002305,2058.578052,6.272625e+05,4733.027741,4.628048e+06,0.262384,0.086177,2402.409074,463610.493671,002-혼자서 걸어요 (Prod. by 나얼)-태연 (TAEYEON).mp3
3,003-Say I Love You-우디(Woody).mp3,135.999178,0.064676,0.060493,0.258724,0.011677,0.000008,0.045789,0.000040,0.015676,1846.858553,8.839437e+05,4357.088485,6.049626e+06,0.277907,0.092972,2422.179305,630201.733698,003-Say I Love You-우디(Woody).mp3
4,004-그댄 행복에 살텐데 (2022)-최유리.mp3,123.046875,0.069991,0.065092,0.182921,0.009579,0.000002,0.028329,0.000100,0.006010,2086.780315,9.046060e+05,5043.194386,6.355688e+06,0.270101,0.088584,2555.323454,655405.712739,004-그댄 행복에 살텐데 (2022)-최유리.mp3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795-THREAT-Rex Orange County(렉스 오렌지 카운티).mp3,123.046875,0.069507,0.064676,0.209674,0.014744,-0.000026,0.030200,-0.000203,0.015764,1874.757983,6.714360e+05,4078.729026,3.709489e+06,0.320778,0.092743,2284.560687,261333.076897,795-THREAT-Rex Orange County(렉스 오렌지 카운티).mp3
796,796-I Don’t Think That I Like Her-Charlie Puth...,92.285156,0.091468,0.083101,0.118296,0.003319,-0.000028,0.008724,0.000037,0.005117,2203.390061,9.052152e+05,4936.624748,4.389698e+06,0.308990,0.095172,2489.020159,362329.549022,796-I Don’t Think That I Like Her-Charlie Puth...
797,797-IT'S YOU (feat. keshi)-MAX(맥스).mp3,89.102909,0.063190,0.059197,0.175053,0.011764,-0.000004,0.027746,-0.000044,0.008641,1865.459667,1.091837e+06,4003.451802,5.192599e+06,0.272719,0.094683,2344.892669,317645.147335,797-IT'S YOU (feat. keshi)-MAX(맥스).mp3
798,798-Nancy Got A Haircut-Alec Benjamin(알렉 벤자민).mp3,151.999081,0.065284,0.061022,0.132996,0.003326,-0.000010,0.012361,-0.000181,0.003733,1800.273524,9.528751e+05,3853.898289,5.142723e+06,0.323034,0.092255,2247.136699,448555.691880,798-Nancy Got A Haircut-Alec Benjamin(알렉 벤자민).mp3


In [9]:
mfcc_arr1 = np.array(mfcc_list1).reshape(-1, 2)
mfcc_arr2 = np.array(mfcc_list2).reshape(-1, 2)

# DataFrame으로 변환
df_mfcc = pd.DataFrame(np.concatenate((mfcc_arr1, mfcc_arr2), axis=1))

# 각 컬럼에 대한 이름 설정
col_names = ['mfcc_mean_'+str(i+1) for i in range(2)]
col_names += ['mfcc_var_'+str(i+1) for i in range(2)]
df_mfcc.columns = col_names

In [10]:
df_mfcc

,mfcc_mean_1,mfcc_mean_2,mfcc_var_1,mfcc_var_2
0,0.533557,0.518298,0.033481,0.032178
1,0.511398,0.502652,0.028534,0.027915
2,0.499789,0.500008,0.031132,0.034356
3,0.507834,0.520479,0.031574,0.024687
4,0.485597,0.499801,0.032763,0.029747
...,...,...,...,...
795,0.540166,0.477447,0.032096,0.033156
796,0.502934,0.522011,0.028053,0.031959
797,0.502735,0.500174,0.030106,0.028842
798,0.502302,0.507481,0.027941,0.026322


In [11]:
df = pd.concat([dfs_df, df_mfcc], axis=1)

In [12]:
df

,filename,tempo1,zero_crossing_rate_mean,zero_crossing_rate_var,rms_mean,rms_var,y_harm_mean,y_harm_var,y_perc_mean,y_perc_var,...,rolloff_var,chroma_stft_mean,chroma_stft_var,spectral_bandwidth_mean,spectral_bandwidth_var,label,mfcc_mean_1,mfcc_mean_2,mfcc_var_1,mfcc_var_2
0,000-너의 모든 순간-성시경.mp3,135.999178,0.063051,0.059076,0.206933,0.014392,0.000041,0.042217,0.000158,0.005620,...,2.830697e+06,0.266660,0.090366,1802.617495,273552.279734,000-너의 모든 순간-성시경.mp3,0.533557,0.518298,0.033481,0.032178
1,001-Monologue-테이(Tei).mp3,78.302557,0.123248,0.108058,0.219556,0.010052,0.000003,0.033471,0.000331,0.011100,...,2.592118e+06,0.344431,0.088892,2412.710045,167487.247350,001-Monologue-테이(Tei).mp3,0.511398,0.502652,0.028534,0.027915
2,002-혼자서 걸어요 (Prod. by 나얼)-태연 (TAEYEON).mp3,135.999178,0.081354,0.074735,0.145015,0.006357,0.000002,0.020258,-0.000002,0.002305,...,4.628048e+06,0.262384,0.086177,2402.409074,463610.493671,002-혼자서 걸어요 (Prod. by 나얼)-태연 (TAEYEON).mp3,0.499789,0.500008,0.031132,0.034356
3,003-Say I Love You-우디(Woody).mp3,135.999178,0.064676,0.060493,0.258724,0.011677,0.000008,0.045789,0.000040,0.015676,...,6.049626e+06,0.277907,0.092972,2422.179305,630201.733698,003-Say I Love You-우디(Woody).mp3,0.507834,0.520479,0.031574,0.024687
4,004-그댄 행복에 살텐데 (2022)-최유리.mp3,123.046875,0.069991,0.065092,0.182921,0.009579,0.000002,0.028329,0.000100,0.006010,...,6.355688e+06,0.270101,0.088584,2555.323454,655405.712739,004-그댄 행복에 살텐데 (2022)-최유리.mp3,0.485597,0.499801,0.032763,0.029747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795-THREAT-Rex Orange County(렉스 오렌지 카운티).mp3,123.046875,0.069507,0.064676,0.209674,0.014744,-0.000026,0.030200,-0.000203,0.015764,...,3.709489e+06,0.320778,0.092743,2284.560687,261333.076897,795-THREAT-Rex Orange County(렉스 오렌지 카운티).mp3,0.540166,0.477447,0.032096,0.033156
796,796-I Don’t Think That I Like Her-Charlie Puth...,92.285156,0.091468,0.083101,0.118296,0.003319,-0.000028,0.008724,0.000037,0.005117,...,4.389698e+06,0.308990,0.095172,2489.020159,362329.549022,796-I Don’t Think That I Like Her-Charlie Puth...,0.502934,0.522011,0.028053,0.031959
797,797-IT'S YOU (feat. keshi)-MAX(맥스).mp3,89.102909,0.063190,0.059197,0.175053,0.011764,-0.000004,0.027746,-0.000044,0.008641,...,5.192599e+06,0.272719,0.094683,2344.892669,317645.147335,797-IT'S YOU (feat. keshi)-MAX(맥스).mp3,0.502735,0.500174,0.030106,0.028842
798,798-Nancy Got A Haircut-Alec Benjamin(알렉 벤자민).mp3,151.999081,0.065284,0.061022,0.132996,0.003326,-0.000010,0.012361,-0.000181,0.003733,...,5.142723e+06,0.323034,0.092255,2247.136699,448555.691880,798-Nancy Got A Haircut-Alec Benjamin(알렉 벤자민).mp3,0.502302,0.507481,0.027941,0.026322


In [13]:
df.to_csv("testdate.csv", index = False)

In [14]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns   # 얘는 내장라이브러리 인듯
import pandas as pd

In [15]:
final = pd.read_csv('testdate.csv', index_col='filename')

labels = final['label']
final = final.drop(columns=['label'])

final_scaled = sklearn.preprocessing.scale(final)
final = pd.DataFrame(final_scaled, columns=final.columns)
final.head()

from sklearn.metrics.pairwise import cosine_similarity 
similarity = cosine_similarity(final)
sim_final = pd.DataFrame(similarity, index=labels.index, columns=labels.index)

def find_similar_songs(name, n=3):
    series = sim_final[name].sort_values(ascending=False)
    series = series.drop(name)
    return series.head(n).to_frame()
    # 매개변수에 들어갈 친구와 가장 비슷한 노래 5곡을 추천해 달라는 함수
    # 여기서 n은 강사님들한테 받은 top_n 이랑 같은 부분이고 name에는 저장된 곡명을 넣으면 된다.
    


In [16]:
sim_final

filename,000-너의 모든 순간-성시경.mp3,001-Monologue-테이(Tei).mp3,002-혼자서 걸어요 (Prod. by 나얼)-태연 (TAEYEON).mp3,003-Say I Love You-우디(Woody).mp3,004-그댄 행복에 살텐데 (2022)-최유리.mp3,005-해요 (2022)-#안녕.mp3,006-마지막 사랑-신예영.mp3,007-다정히 내 이름을 부르면-경서예지.mp3,008-그때 그 순간 그대로 (그그그)-WSG워너비 (가야G).mp3,"009-모든 날, 모든 순간 (Every day, Every Moment)-폴킴(Paul Kim).mp3",...,790-Someone Like You-Adele(아델).mp3,791-Be So Cruel-ASTN.mp3,792-High-5 Seconds Of Summer(파이브 세컨즈 오브 썸머).mp3,793-Be Alright-John K.mp3,794-Sugar-Maroon 5(마룬 파이브).mp3,795-THREAT-Rex Orange County(렉스 오렌지 카운티).mp3,796-I Don’t Think That I Like Her-Charlie Puth(찰리 푸스).mp3,797-IT'S YOU (feat. keshi)-MAX(맥스).mp3,798-Nancy Got A Haircut-Alec Benjamin(알렉 벤자민).mp3,799-Bonfire-Peder Elias.mp3
filename,,,,,,,,,,,,,,,,,,,,,
000-너의 모든 순간-성시경.mp3,1.000000,-0.391866,0.388633,0.161225,0.118690,0.906056,0.781650,0.025541,-0.595959,0.581798,...,0.494869,-0.100484,0.220525,-0.096230,-0.180968,0.666086,-0.099738,0.336688,0.171570,-0.080668
001-Monologue-테이(Tei).mp3,-0.391866,1.000000,-0.362282,-0.635398,-0.546509,-0.365815,-0.362757,-0.338656,0.519014,-0.342647,...,-0.426586,-0.405829,0.082770,-0.231331,0.139896,-0.385761,-0.016405,-0.389854,-0.609374,-0.407122
002-혼자서 걸어요 (Prod. by 나얼)-태연 (TAEYEON).mp3,0.388633,-0.362282,1.000000,0.096418,0.676646,0.216562,0.592975,0.238828,-0.294211,0.558861,...,0.682611,0.143413,0.175626,-0.523694,0.032319,0.370698,0.555030,0.301381,0.409713,-0.209639
003-Say I Love You-우디(Woody).mp3,0.161225,-0.635398,0.096418,1.000000,0.698871,0.087570,0.223014,0.738676,-0.209189,0.152133,...,0.293083,0.342091,0.248948,0.346693,-0.224882,0.058623,-0.117263,0.502065,0.476487,0.427970
004-그댄 행복에 살텐데 (2022)-최유리.mp3,0.118690,-0.546509,0.676646,0.698871,1.000000,-0.073406,0.420076,0.648058,-0.152585,0.381869,...,0.547893,0.426968,0.319124,-0.142531,-0.157317,0.176513,0.361451,0.574251,0.525540,0.173420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795-THREAT-Rex Orange County(렉스 오렌지 카운티).mp3,0.666086,-0.385761,0.370698,0.058623,0.176513,0.592798,0.630112,-0.197964,-0.404583,0.361201,...,0.238035,0.099219,0.205611,0.222218,-0.240921,1.000000,-0.013759,0.407687,-0.000482,0.080159
796-I Don’t Think That I Like Her-Charlie Puth(찰리 푸스).mp3,-0.099738,-0.016405,0.555030,-0.117263,0.361451,-0.155019,-0.071908,-0.166206,-0.158120,0.511605,...,0.449018,0.582686,-0.002302,-0.220218,0.649172,-0.013759,1.000000,0.431408,0.441696,0.225944
797-IT'S YOU (feat. keshi)-MAX(맥스).mp3,0.336688,-0.389854,0.301381,0.502065,0.574251,0.291882,0.362310,0.159383,-0.317007,0.577925,...,0.474729,0.628496,0.305450,0.210136,-0.027146,0.407687,0.431408,1.000000,0.524168,0.480040


In [24]:
sim_final.iloc[1][:10]

filename
000-너의 모든 순간-성시경.mp3                                         -0.391866
001-Monologue-테이(Tei).mp3                                     1.000000
002-혼자서 걸어요 (Prod. by 나얼)-태연 (TAEYEON).mp3                   -0.362282
003-Say I Love You-우디(Woody).mp3                             -0.635398
004-그댄 행복에 살텐데 (2022)-최유리.mp3                                -0.546509
005-해요 (2022)-#안녕.mp3                                        -0.365815
006-마지막 사랑-신예영.mp3                                           -0.362757
007-다정히 내 이름을 부르면-경서예지.mp3                                   -0.338656
008-그때 그 순간 그대로 (그그그)-WSG워너비 (가야G).mp3                        0.519014
009-모든 날, 모든 순간 (Every day, Every Moment)-폴킴(Paul Kim).mp3   -0.342647
Name: 001-Monologue-테이(Tei).mp3, dtype: float64

In [25]:
find_similar_songs('009-모든 날, 모든 순간 (Every day, Every Moment)-폴킴(Paul Kim).mp3')

,"009-모든 날, 모든 순간 (Every day, Every Moment)-폴킴(Paul Kim).mp3"
filename,
069-너를 만나-폴킴(Paul Kim).mp3,0.921531
508-우리가 맞다는 대답을 할 거예요 (@leekangseung111)-10CM.mp3,0.921084
238-비-폴킴(Paul Kim).mp3,0.919005
